In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

ds = pd.read_csv("AR_all.csv")
ds=ds[0:11]
print(ds[[ 'name','id','sku']])

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), min_df=1, stop_words='english')
tfidf_matrix = tf.fit_transform(ds['name'])
print(tf.get_feature_names())
#print(tfidf_matrix)
#print(ds['name'][0:5])
tf1 = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), min_df=1, stop_words='english')
tfidf_matrix1 = tf1.fit_transform(ds['sku'])
print(tf1.get_feature_names())
#print(tfidf_matrix1)

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_similarities1 = linear_kernel(tfidf_matrix1, tfidf_matrix1)
#print(cosine_similarities)
results = {}
results1 = {}

for idx, row in ds.iterrows():
    #print('',)
    similar_indices = cosine_similarities[idx].argsort()[::-1]
   # print(similar_indices)
    similar_items = [(cosine_similarities[idx][i], ds['id'][i]) for i in similar_indices]
    similar_indices1 = cosine_similarities1[idx].argsort()[::-1]
    #print(similar_indices1)
    similar_items1 = [(cosine_similarities1[idx][i], ds['id'][i]) for i in similar_indices1]
    #print(similar_items1)
    # First item is the item itself, so remove it.
    # Each dictionary entry is like: [(1,2), (3,4)], with each tuple being (score, item_id)
    results[row['id']] = similar_items[1:]
    results1[row['id']] = similar_items1[1:]  
print('done!')

                                                name  id  \
0   Coffee coco butter ylang ylang soap 125gm (Copy)   4   
1        OW' Real Tulsi Green Tea + Saffron -25 bags   5   
2      Aum Fresh Sundarikalp Tea 50 gm (25 Tea Bags)  10   
3                  Baby natural laundry wash 1000 ml  11   
4            Kairali Amruthotharam Kashayam (200 ml)  12   
5         Aum Fresh Nidrakar Tea 50 gm (25 Tea Bags)  13   
6           Aum Fresh Memory Tea 50 gm (25 Tea Bags)  14   
7           Kairali Balaguluchyedi Kashayam (200 ml)  15   
8       Aum Fresh Gas Relief Tea 50 gm (25 Tea Bags)  16   
9         Aum Fresh Vajikaran Tea 50gm (25 Tea Bags)  17   
10         Kairali Cheriya Rasnadi Kashayam (200 ml)  18   

                                              sku  
0                                        SKU 9480  
1                                       SKU 11067  
2   Aum Fresh Sundarikalp Tea 50 gm (25 Tea Bags)  
3               Baby natural laundry wash 1000 ml  
4         Kairali A

In [4]:
def item(id):
    return ds.loc[ds['id'] == id]['name'].tolist()[0].split(' - ')[0]

# Just reads the results out of the dictionary. No real logic here.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    recs1 = results1[item_id][:num]
    print(recs1)
    l=[]
    l1=[]
    #Sprint(recs)
    for i in recs:
        l.append(i[1])
    print(l)     
    for rec in recs:
       if rec[0]>(0.50/2):
         print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")
    for i in recs1:
        l1.append(i[1])
    print(l1)     
    for rec in recs1:
       if rec[0]>0.50:
         print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")    

# Just plug in any item id here (1-500), and the number of recommendations you want (1-99)
# You can get a list of valid item IDs by evaluating the variable 'ds', or a few are listed below

recommend(item_id=10, num=10)

Recommending 10 products similar to Aum Fresh Sundarikalp Tea 50 gm (25 Tea Bags)...
-------
[(0.7936077671566899, 14), (0.7936077671566899, 13), (0.7225395819469875, 16), (0.601304817994677, 17), (0.0, 18), (0.0, 15), (0.0, 12), (0.0, 11), (0.0, 5), (0.0, 4)]
[14, 13, 16, 17, 5, 18, 15, 12, 11, 4]
Recommended: Aum Fresh Memory Tea 50 gm (25 Tea Bags) (score:0.775864421749223)
Recommended: Aum Fresh Nidrakar Tea 50 gm (25 Tea Bags) (score:0.775864421749223)
Recommended: Aum Fresh Gas Relief Tea 50 gm (25 Tea Bags) (score:0.7012470979466564)
Recommended: Aum Fresh Vajikaran Tea 50gm (25 Tea Bags) (score:0.5674002443191339)
[14, 13, 16, 17, 18, 15, 12, 11, 5, 4]
Recommended: Aum Fresh Memory Tea 50 gm (25 Tea Bags) (score:0.7936077671566899)
Recommended: Aum Fresh Nidrakar Tea 50 gm (25 Tea Bags) (score:0.7936077671566899)
Recommended: Aum Fresh Gas Relief Tea 50 gm (25 Tea Bags) (score:0.7225395819469875)
Recommended: Aum Fresh Vajikaran Tea 50gm (25 Tea Bags) (score:0.601304817994677)
